In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import sklearn
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import inspect

In [5]:
config = {"input_size" : 784,
          "hidden_layer1" : 512,
          "hidden_layer2" : 256,
          "bottleneck" : 128,
          "hidden_layer3" : 256,
          "hidden_layer4" : 512,
          "learning_rate" : 0.001,
          "number_epoch" : 100,
          "batch_size" : 100
          }

In [3]:
transformation = transforms.Compose([transforms.ToTensor()])

In [ ]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transformation, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transformation, download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=config["batch_size"], shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=config["batch_size"], shuffle=False)

In [ ]:
class VariationalAutoencoder(nn.Module):
    def __init__(self, input_size, hidden_layer1, hidden_layer2, bottleneck, hidden_layer3, hidden_layer4, mu):
        super(VariationalAutoencoder, self).__init__()

        self.encoding_layers = nn.Sequential(
            nn.Linear(input_size, hidden_layer1),
            nn.ReLU(),
            nn.Linear(hidden_layer1, hidden_layer2),
            nn.ReLU(),
            nn.Linear(hidden_layer2, bottleneck)
        )

        self.decoding_layers = nn.Sequential(
            nn.Linear(bottleneck, hidden_layer3),
            nn.ReLU(),
            nn.Linear(hidden_layer3, hidden_layer4),
            nn.ReLU(),
            nn.Linear(hidden_layer4, input_size)
        )  
    def encoder_forward(self)